In [2]:
import numpy as np
from random import shuffle
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [141]:
n_instancias=10
n_caracteristicas=8
n_clases=5
rnd_neighbors=np.array([])
rnd_dimensions=np.array([])
e=DecisionTreeClassifier()

In [4]:
def conjunto_datos(n_instancias,n_caracteristicas,n_clases):
        return make_multilabel_classification(n_samples=n_instancias, 
                                              n_features=n_caracteristicas, n_classes=n_clases)   

In [105]:
X,Y=conjunto_datos(n_instancias,n_caracteristicas,n_clases)
X

array([[  7.,   4.,   9.,   2.,   6.,   5.,   7.,   5.],
       [  9.,   5.,   9.,  10.,   4.,   7.,   2.,   1.],
       [  5.,   2.,   8.,   8.,   5.,   6.,   4.,  10.],
       [  5.,   4.,   8.,  10.,   5.,   8.,   4.,   7.],
       [  3.,   4.,   8.,   3.,   6.,   5.,   3.,  16.],
       [  2.,   1.,  12.,  17.,   6.,   7.,   4.,   0.],
       [  5.,   2.,   1.,   1.,  10.,   1.,   4.,  13.],
       [ 10.,   5.,   6.,   5.,   6.,   9.,   5.,  13.],
       [ 10.,   6.,   4.,   3.,   7.,   9.,   6.,   4.],
       [  7.,  15.,   7.,   0.,   3.,   9.,   1.,  15.]])

In [106]:
def aleatorio_boolean(n_caracteristicas):
    rnd_dimensions=np.random.randint(0, 2,n_caracteristicas)
    return rnd_dimensions.astype(bool)  

In [107]:
def aleatorio_array(n_instancias):
    s=list(range(n_instancias))
    shuffle(s)
    return np.array(s[:(int(len(s)/2))])

In [108]:
def reducir_datos(X,rnd_dimensions):
    return X[:, rnd_dimensions]

In [109]:
def vecino_cercano(n_instancias,rnd_neighbors,m_reducida):
    m_vecinos=np.zeros((n_instancias,len(rnd_neighbors)))
    cont=-1
    for i in m_reducida:
        dist=999
        cont+=1
        cont2=-1
        for j in rnd_neighbors:
            cont2+=1
            dist2=euclidean_distances([i],[m_reducida[j,:]])
            if dist2<dist:
                dist=dist2
                a=cont
                b=cont2
        m_vecinos[a][b]=1
    return m_vecinos

In [142]:
def clasificador(X,Y,m_vecinos):
    m_entrenamiento=np.concatenate((X,m_vecinos),axis=1)
    return e.fit(m_entrenamiento,Y)

In [133]:
def fit(X,Y,n_instancias):
    rnd_dimensions=aleatorio_boolean(n_caracteristicas)
    rnd_neighbors=aleatorio_array(n_instancias)
    m_reducida=reducir_datos(X,rnd_dimensions)
    m_vecinos=vecino_cercano(n_instancias,rnd_neighbors,m_reducida)
    return clasificador(X,Y,m_vecinos),rnd_neighbors,rnd_dimensions

In [143]:
arbol,rnd_neighbors,rnd_dimensions=fit(X,Y,n_instancias)
print(rnd_neighbors)

[[  4.   9.   2.   6.   5.   7.]
 [  5.   9.  10.   4.   7.   2.]
 [  2.   8.   8.   5.   6.   4.]
 [  4.   8.  10.   5.   8.   4.]
 [  4.   8.   3.   6.   5.   3.]
 [  1.  12.  17.   6.   7.   4.]
 [  2.   1.   1.  10.   1.   4.]
 [  5.   6.   5.   6.   9.   5.]
 [  6.   4.   3.   7.   9.   6.]
 [ 15.   7.   0.   3.   9.   1.]]
[2 4 1 0 6]


In [144]:
with open("mi_arbol.dot", 'w') as archivo_dot:
    tree.export_graphviz(arbol, out_file = archivo_dot)

In [145]:
!dot -Tpng mi_arbol.dot -o arbol_decision.png

In [146]:
X1,Y1=conjunto_datos(n_instancias,n_caracteristicas,n_clases)
X1

array([[  4.,  11.,   4.,   7.,   2.,   8.,   3.,   9.],
       [  5.,   4.,   6.,  12.,   8.,  11.,   1.,   9.],
       [  8.,   9.,   8.,   7.,   6.,   7.,   7.,   8.],
       [  9.,   6.,   2.,   4.,   2.,   5.,   5.,   6.],
       [  4.,  11.,   3.,   9.,   3.,  10.,   3.,   6.],
       [  6.,  16.,   7.,   0.,   4.,   5.,  10.,   1.],
       [  2.,   7.,   6.,   9.,   1.,   6.,   6.,   9.],
       [  2.,   3.,   1.,  10.,   1.,  14.,   1.,   2.],
       [ 11.,   8.,   4.,   3.,   3.,   4.,  11.,   2.],
       [  9.,  10.,   6.,   5.,   2.,  10.,   6.,   2.]])

In [147]:
m_predecir=np.zeros((len(rnd_neighbors),n_caracteristicas))
cont=0
for i in range(len(X1)):
    if i in rnd_neighbors:
        m_predecir[cont]=X1[i,:]
        cont+=1
m_predecir

array([[  4.,  11.,   4.,   7.,   2.,   8.,   3.,   9.],
       [  5.,   4.,   6.,  12.,   8.,  11.,   1.,   9.],
       [  8.,   9.,   8.,   7.,   6.,   7.,   7.,   8.],
       [  4.,  11.,   3.,   9.,   3.,  10.,   3.,   6.],
       [  2.,   7.,   6.,   9.,   1.,   6.,   6.,   9.]])

In [151]:
m_reducida2=reducir_datos(X1,rnd_dimensions)

In [149]:
m_vecinos2=vecino_cercano(n_instancias,rnd_neighbors,m_reducida2)
m_entrenamiento2=np.concatenate((X1,m_vecinos2),axis=1)

In [150]:
e.predict(m_entrenamiento2)
print(e)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
